In [86]:
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
from datetime import datetime, date, timedelta
import pandas as pd
from pandas import DataFrame as df

In [87]:
# need to fdownload
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [88]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [89]:
def date_to_string(date):
    month, day = date.month, date.day
    if(date.month < 10):
        month = "0"+str(month)
    if date.day < 10:
        day = "0"+str(day)
    return "{}-{}-{}".format(date.year, month, day)

In [92]:
def get_keyword_df(dataframe, start_date, end_date):
    start_time = datetime.strptime(start_date, "%Y-%m-%d")
    last_time = datetime.strptime(end_date, "%Y-%m-%d")
    while last_time >= start_time:
        date_df = dataframe[dataframe['date'] == date_to_string(last_time)]
        
        if date_df.empty:
            last_time -= timedelta(days = 1)
            continue
        
        print(date_to_string(last_time))
        titles = date_df['title']
        tokens = []
        sentences = ""
        for title in titles:
            sentence = title.strip().lower()
            token = word_tokenize(sentence)
            filtered_token = [lemmatizer.lemmatize(w) for w in token if not w in stop_words and w not in ".,/:'\"’‘$%"]
            tokens += filtered_token

        freq_dict = {}
        for word in tokens:
            if word in freq_dict:
                freq_dict[word] += 1
            else:
                freq_dict[word] = 1

        freq_list = list(freq_dict.items())
        freq_list.sort(key = lambda x : x[1], reverse = True)
        print(freq_list[:10])
        last_time -= timedelta(days = 1)
            
    
    return dataframe

In [93]:
def save_df(dataframe, start_date, end_date, output_dir):
    name = "news_keyword_{}_{}.csv".format(start_date, end_date)
    dataframe.to_csv(output_dir + "/" + name)

In [94]:
def get_and_save_keyword(start_date, end_date, output_dir = "../cleaned_data/news/"):
    input_df = pd.read_csv(output_dir+"news_all_{}_{}.csv".format(start_date, end_date))
    output_df = get_keyword_df(input_df, start_date, end_date)
    if "Unnamed: 0" in output_df:
        del output_df["Unnamed: 0"]
    save_df(output_df, start_date, end_date, output_dir)
    

In [95]:
get_and_save_keyword("2022-1-1", "2022-12-31")

2022-03-24
[('crypto', 3), ('project', 2), ('exchange', 2), ('one', 2), ('pyramid', 1), ('busted', 1), ('russia', 1), ('loss', 1), ('exceed', 1), ('10', 1)]
2022-03-23
[('launch', 2), ('bitcoin', 2), ('bond', 2), ('crypto', 2), ('iexec', 2), ('gamestop', 1), ('beta', 1), ('nft', 1), ('marketplace', 1), ('platform', 1)]
2022-03-22
[('crypto', 5), ('list', 2), ('mining', 2), ('new', 2), ("'s", 2), ('japanese', 1), ('exchange', 1), ('coincheck', 1), ('nasdaq', 1), ('via', 1)]
2022-03-21
[('crypto', 3), ('price', 3), ('service', 2), ('cardano', 2), ('poised', 2), ('pull', 2), ('mind-blowing', 2), ('rally', 2), ('developer', 2), ('look', 2)]
2022-03-20
[('crypto', 3), ('wallet', 2), ('price', 2), ('dogecoin', 1), ('good', 1), ('entry', 1), ('trigger', 1), ('doge', 1), ('bull', 1), ('found', 1)]
2022-03-19
[('crypto', 3), ('bitcoin', 3), ('price', 2), ('hit', 2), ('1inch', 1), ('break', 1), ('past', 1), ('march', 1), ('swing', 1), ('high', 1)]
2022-03-18
[('crypto', 3), ('optimism', 1), ('ra

AttributeError: 'float' object has no attribute 'strip'

In [29]:
d = datetime.strptime("2022-01-01", "%Y-%m-%d")
d

datetime.datetime(2022, 1, 1, 0, 0)